## Enviorment

In [24]:
#verify user
!whoami
#verify user account
!gcloud auth list
#which enciorment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

desktop-2qv6e1h\flash
      Credentialed Accounts
ACTIVE  ACCOUNT
*       michael.zl.prime@gmail.com
Python running from: c:\program files\python36-64
Current working dir C:\Users\flash\Project\Yet-Another-Openpose-Implmentation



To set the active account, run:
    $ gcloud config set account `ACCOUNT`



In [25]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all --purge_orphaned_data True > /tmp/tensorboard_run.log

---

In [26]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 

import ipywidgets as widgets
import datetime
import tensorflow as tf

from models.six_stage_linear_model import ModelMaker
import callbacks
import dataset_builder
from config import *
import load_weights

print(tf.version.VERSION)

2.0.0


In [27]:
if TPU_MODE:
    import tpu_training.test_connectivity
    import tpu_training.init_tpu as init_tpu
    strategy=init_tpu.init_tpu()

---
# Make dataset

In [28]:
tfrecord_files_train,tfrecord_files_val=dataset_builder.get_tfrecord_filenames()
print("Found the following TFrecords:\n","\n".join(tfrecord_files_train),"\n", "\n".join(tfrecord_files_val))
dst=dataset_builder.build_training_ds(tfrecord_files_train)
dsv=dataset_builder.build_validation_ds(tfrecord_files_val)

Retrieving TFrecords in local mode
Found the following TFrecords:
 .\dataset\transformed\person_keypoints_train2017-001.tfrecords
.\dataset\transformed\person_keypoints_train2017-002.tfrecords
.\dataset\transformed\person_keypoints_train2017-003.tfrecords
.\dataset\transformed\person_keypoints_train2017-004.tfrecords
.\dataset\transformed\person_keypoints_train2017-005.tfrecords
.\dataset\transformed\person_keypoints_train2017-006.tfrecords
.\dataset\transformed\person_keypoints_train2017-007.tfrecords
.\dataset\transformed\person_keypoints_train2017-008.tfrecords
.\dataset\transformed\person_keypoints_train2017-009.tfrecords
.\dataset\transformed\person_keypoints_train2017-010.tfrecords
.\dataset\transformed\person_keypoints_train2017-011.tfrecords 
 .\dataset\transformed\person_keypoints_val2017-001.tfrecords
.\dataset\transformed\person_keypoints_val2017-002.tfrecords
.\dataset\transformed\person_keypoints_val2017-003.tfrecords


In [29]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

---
# Callbacks

In [30]:
fit_callbacks=[
    callbacks.checkpoints_callback
    ,callbacks.tensorboard_callback
    ,callbacks.learning_rate_scheduler_callback
    ,callbacks.print_lr_callback
    ,tf.keras.callbacks.TerminateOnNaN()
]


In [31]:
#run to clean tensorboard results
#!gsutil -m rm -r {TENSORBOARD_PATH}
#!gsutil -m rm -r {CHECKPOINTS_PATH}



---
# Model
## Load weights

In [32]:
checkpoint,starting_epoch=load_weights.checkpoints_prompt()


Found these checkpoints
0.Dont load checkpoint


In [33]:
model_maker=ModelMaker() #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics={'stage1paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage2paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage3paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage4paf_output':  [tf.keras.metrics.MeanAbsoluteError()]
         ,'stage5heatmap_output': [
             #AnalogRecall(),
             tf.keras.metrics.MeanAbsoluteError()]    
         ,'stage6heatmap_output': [
             #AnalogRecall(),
             tf.keras.metrics.MeanAbsoluteError()]
        }
    
    train_model.compile(optimizer=tf.keras.optimizers.Adam()
                    ,loss=tf.keras.losses.MeanSquaredError(name="MSE_loss")
                    ,metrics=metrics                    
                   )
    return train_model,test_model

if TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

---
Actually training

In [ ]:
train_history=train_model.fit(
    dst
    ,epochs=TRAINING_EPOCHS
    ,steps_per_epoch=STEPS_PER_EPOCH
    ,validation_data=dsv
    ,callbacks=fit_callbacks
    ,initial_epoch=starting_epoch
)

Train for 28000 steps

Learning rate for epoch 0 is 0.00019999999494757503
Epoch 1/100
   20/28000 [..............................] - ETA: 145:47:58 - loss: 0.0889 - stage1paf_output_loss: 0.0027 - stage2paf_output_loss: 0.0028 - stage3paf_output_loss: 0.0026 - stage4paf_output_loss: 0.0012 - stage5heatmap_output_loss: 0.0394 - stage6heatmap_output_loss: 0.0403 - stage1paf_output_mean_absolute_error: 0.0086 - stage2paf_output_mean_absolute_error: 0.0091 - stage3paf_output_mean_absolute_error: 0.0074 - stage4paf_output_mean_absolute_error: 0.0044 - stage5heatmap_output_mean_absolute_error: 0.0977 - stage6heatmap_output_mean_absolute_error: 0.0905

In [ ]:
model_path=MODELS_PATH+datetime.datetime.now().strftime("%d(%a)%m%y-%H%M")
tf.keras.models.save_model(train_model,model_path)


In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data